<center><img src="car.jpg" width=500></center>


Insurance companies invest a lot of time and money into optimizing their pricing and accurately estimating the likelihood that customers will make a claim. In many countries insurance it is a legal requirement to have car insurance in order to drive a vehicle on public roads, so the market is very large!

(`Source: https://www.accenture.com/_acnmedia/pdf-84/accenture-machine-leaning-insurance.pdf`) 

Knowing all of this, On the Road car insurance have requested your services in building a model to predict whether a customer will make a claim on their insurance during the policy period. As they have very little expertise and infrastructure for deploying and monitoring machine learning models, they've asked you to identify the single feature that results in the best performing model, as measured by accuracy, so they can start with a simple model in production.

They have supplied you with their customer data as a csv file called `car_insurance.csv`, along with a table detailing the column names and descriptions below.



## The dataset

| Column | Description |
|--------|-------------|
| `id` | Unique client identifier |
| `age` | Client's age: <br> <ul><li>`0`: 16-25</li><li>`1`: 26-39</li><li>`2`: 40-64</li><li>`3`: 65+</li></ul> |
| `gender` | Client's gender: <br> <ul><li>`0`: Female</li><li>`1`: Male</li></ul> |
| `driving_experience` | Years the client has been driving: <br> <ul><li>`0`: 0-9</li><li>`1`: 10-19</li><li>`2`: 20-29</li><li>`3`: 30+</li></ul> |
| `education` | Client's level of education: <br> <ul><li>`0`: No education</li><li>`1`: High school</li><li>`2`: University</li></ul> |
| `income` | Client's income level: <br> <ul><li>`0`: Poverty</li><li>`1`: Working class</li><li>`2`: Middle class</li><li>`3`: Upper class</li></ul> |
| `credit_score` | Client's credit score (between zero and one) |
| `vehicle_ownership` | Client's vehicle ownership status: <br><ul><li>`0`: Does not own their vehilce (paying off finance)</li><li>`1`: Owns their vehicle</li></ul> |
| `vehcile_year` | Year of vehicle registration: <br><ul><li>`0`: Before 2015</li><li>`1`: 2015 or later</li></ul> |
| `married` | Client's marital status: <br><ul><li>`0`: Not married</li><li>`1`: Married</li></ul> |
| `children` | Client's number of children |
| `postal_code` | Client's postal code | 
| `annual_mileage` | Number of miles driven by the client each year |
| `vehicle_type` | Type of car: <br> <ul><li>`0`: Sedan</li><li>`1`: Sports car</li></ul> |
| `speeding_violations` | Total number of speeding violations received by the client | 
| `duis` | Number of times the client has been caught driving under the influence of alcohol |
| `past_accidents` | Total number of previous accidents the client has been involved in |
| `outcome` | Whether the client made a claim on their car insurance (response variable): <br><ul><li>`0`: No claim</li><li>`1`: Made a claim</li></ul> |

In [87]:
# https://projects.datacamp.com/projects/1645

# Import required modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.api.types import is_numeric_dtype
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    classification_report
)

# Import CSV Data
df = pd.read_csv('car_insurance.csv')

# Review Data
# df.info() # Get a summary of the DataFrame, including data types and non-null values
# df.head() # View the first few rows of the DataFrame
# df.describe() # Get descriptive statistics for numerical columns

# -- Check for Imbalanced Data
class_counts = df['outcome'].value_counts()
class_proportions = df['outcome'].value_counts(normalize=True) * 100
# print("Class Counts:\n", class_counts)
# print("\nClass Proportions (%):\n", class_proportions)
# plt.figure(figsize=(8, 6))
# sns.countplot(x='outcome', data=df)
# plt.title('Distribution of Outcome Classes')
# plt.xlabel('Class')
# plt.ylabel('Count')
# plt.show()

# -- Plot Data
# for col in df.columns:
#     if is_numeric_dtype(df[col]):
#         plt.hist(df[[col]])
#         plt.xlabel(col)
#         plt.ylabel("Frequency")
#         plt.show()


# -- Objects: driving_experience, education, income, vehicle_year, vehicle_type
# -- Missing Data: credit_score, anual_mileage

# Clean / Transform Data
# -- Convert Objects to Numbers
# -- Convert object type to a categorical type
df['driving_experience'] = df['driving_experience'].astype('category')
df['education'] = df['education'].astype('category')
df['income'] = df['income'].astype('category')
df['vehicle_year'] = df['vehicle_year'].astype('category')
df['vehicle_type'] = df['vehicle_type'].astype('category')
# -- Convert the categorical column to numerical codes
df['driving_experience'] = df['driving_experience'].cat.codes
df['education'] = df['education'].cat.codes
df['income'] = df['income'].cat.codes
df['vehicle_year'] = df['vehicle_year'].cat.codes
df['vehicle_type'] = df['vehicle_type'].cat.codes

# -- Fill missing values with a specific value (e.g., mean, median, or a constant)
df['credit_score'].fillna(df['credit_score'].mean(), inplace=True)
df['annual_mileage'].fillna(df['annual_mileage'].mean(), inplace=True)

# -- Handle Duplicates (None)
# df.drop_duplicates(inplace=True)

# -- Review Data
# df.info() # Get a summary of the DataFrame, including data types and non-null values
# df.head() # View the first few rows of the DataFrame
# df.describe() # Get descriptive statistics for numerical columns

# -- Plot Data
# for col in df.columns:
#     plt.hist(df[[col]])
#     plt.xlabel(col)
#     plt.ylabel("Frequency")
#     plt.show()

# Identify best feature, as measured by accuracy
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter

def find_best_feature(df, model, scale=False, target='outcome', show_confusion=True):

    feature_metrics = []

    # Reuse the same train/test split for fairness across features
    X_full = df.drop(columns=[target,'id'])
    y = df[target]
    # print("Original class distribution:", Counter(y))
    
    # Undersampling using RandomUnderSampler
    undersample = RandomUnderSampler(sampling_strategy='majority')
    X_full, y = undersample.fit_resample(X_full, y)
    # print("Undersampled class distribution:", Counter(y))

    # # Oversampling using RandomOverSampler - Yields Same Overall Results as Undersample
    # oversample = RandomOverSampler(sampling_strategy='minority')
    # X_full, y = oversample.fit_resample(X_full, y)
    # print("Oversampled class distribution:", Counter(y))
    
    X_train_full, X_test_full, y_train, y_test = train_test_split(
        X_full, y, test_size=0.2, random_state=42)

    for col in X_full.columns:
        X_train = X_train_full[[col]].copy()
        X_test = X_test_full[[col]].copy()

        # Optional scaling
        if scale:
            scaler = StandardScaler()
            X_train = scaler.fit_transform(X_train)
            X_test = scaler.transform(X_test)

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Collect metrics
        acc = accuracy_score(y_test, y_pred)
        prec = precision_score(y_test, y_pred, zero_division=0)
        rec = recall_score(y_test, y_pred, zero_division=0)
        f1 = f1_score(y_test, y_pred, zero_division=0)
        unique_preds = np.unique(y_pred, return_counts=True)

        feature_metrics.append({
            'Feature': col,
            'Accuracy': acc,
            'Precision': prec,
            'Recall': rec,
            'F1': f1,
            'Predicted_Classes': dict(zip(unique_preds[0], unique_preds[1]))
        })

    # Convert to DataFrame for readability
    results = pd.DataFrame(feature_metrics).sort_values(by='Accuracy', ascending=False)
    best_feature = results.iloc[0]['Feature']
    best_acc = results.iloc[0]['Accuracy']

    print(f"\n Best Feature: {best_feature} (Accuracy: {best_acc:.4f})")
    print("\n Summary of All Features:\n")
    display(results)

    # Optional confusion matrix for best feature
    if show_confusion:
        print(f"\n🔍 Confusion Matrix for Best Feature: {best_feature}")
        X_train = X_train_full[[best_feature]]
        X_test = X_test_full[[best_feature]]

        if scale:
            scaler = StandardScaler()
            X_train = scaler.fit_transform(X_train)
            X_test = scaler.transform(X_test)

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        cm = confusion_matrix(y_test, y_pred)
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
        plt.xlabel('Predicted')
        plt.ylabel('Actual')
        plt.title(f'Confusion Matrix – {best_feature}')
        plt.show()

        print("\nClassification Report:")
        print(classification_report(y_test, y_pred, zero_division=0))


# -- LogisticRegression
print("Logisitic Regression")
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='liblinear', random_state=42)      
find_best_feature(df, model, scale=False, target='outcome', show_confusion=False)

# -- DecisionTreeClassifier
print("Decision Tree Classifier")
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(criterion='gini', max_depth=10)
find_best_feature(df, model, scale=False, target='outcome', show_confusion=False)

# -- RandomForestClassifier
print("Random Forest Classifier")
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
find_best_feature(df, model, scale=False, target='outcome', show_confusion=False)

# -- Standard Vector Machine
print("Standard Vector Machine Classifier")
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
model = SVC(kernel='linear', C=1.0, random_state=42) # Linear kernel for binary class
find_best_feature(df, model, scale=True, target='outcome', show_confusion=False)

# Results
# -- Original class distribution: Counter({0.0: 6867, 1.0: 3133})
# -- Undersampled class distribution: Counter({0.0: 3133, 1.0: 3133})
# -- LogisticRegression: driving_experience (Accuracy: 0.7560)
# -- DecisionTreeClassifier: driving_experience (Accuracy: 0.7616)
# -- RandomForestClassifier: driving_experience (Accuracy: 0.7544)
# -- Standard Vector Machine: driving_experience (Accuracy: 0.7671)


Logisitic Regression

 Best Feature: driving_experience (Accuracy: 0.7608)

 Summary of All Features:



,Feature,Accuracy,Precision,Recall,F1,Predicted_Classes
2,driving_experience,0.760766,0.767318,0.718802,0.742268,"{0.0: 691, 1.0: 563}"
0,age,0.710526,0.660811,0.813644,0.729306,"{0.0: 514, 1.0: 740}"
15,past_accidents,0.702552,0.645408,0.841930,0.730686,"{0.0: 470, 1.0: 784}"
6,vehicle_ownership,0.681818,0.718615,0.552413,0.624647,"{0.0: 792, 1.0: 462}"
8,married,0.664274,0.631195,0.720466,0.672883,"{0.0: 568, 1.0: 686}"
13,speeding_violations,0.651515,0.593182,0.868552,0.704929,"{0.0: 374, 1.0: 880}"
5,credit_score,0.635566,0.625000,0.599002,0.611725,"{0.0: 678, 1.0: 576}"
7,vehicle_year,0.633971,0.573499,0.921797,0.707084,"{0.0: 288, 1.0: 966}"
9,children,0.633174,0.665105,0.472546,0.552529,"{0.0: 827, 1.0: 427}"
4,income,0.625199,0.613518,0.589018,0.601019,"{0.0: 677, 1.0: 577}"


Decision Tree Classifier

 Best Feature: driving_experience (Accuracy: 0.7568)

 Summary of All Features:



,Feature,Accuracy,Precision,Recall,F1,Predicted_Classes
2,driving_experience,0.756778,0.760563,0.718802,0.739093,"{0.0: 686, 1.0: 568}"
0,age,0.704944,0.654618,0.813644,0.725519,"{0.0: 507, 1.0: 747}"
15,past_accidents,0.700159,0.642948,0.841930,0.729107,"{0.0: 467, 1.0: 787}"
13,speeding_violations,0.698565,0.649664,0.805324,0.719168,"{0.0: 509, 1.0: 745}"
4,income,0.692982,0.696364,0.637271,0.665508,"{0.0: 704, 1.0: 550}"
6,vehicle_ownership,0.670654,0.697479,0.552413,0.616527,"{0.0: 778, 1.0: 476}"
8,married,0.647528,0.612447,0.720466,0.662080,"{0.0: 547, 1.0: 707}"
7,vehicle_year,0.640351,0.578288,0.921797,0.710712,"{0.0: 296, 1.0: 958}"
5,credit_score,0.615630,0.582524,0.698835,0.635401,"{0.0: 533, 1.0: 721}"
9,children,0.611643,0.625551,0.472546,0.538389,"{0.0: 800, 1.0: 454}"


Random Forest Classifier

 Best Feature: driving_experience (Accuracy: 0.7711)

 Summary of All Features:



,Feature,Accuracy,Precision,Recall,F1,Predicted_Classes
2,driving_experience,0.771132,0.785455,0.718802,0.750652,"{0.0: 704, 1.0: 550}"
0,age,0.735247,0.689704,0.813644,0.746565,"{0.0: 545, 1.0: 709}"
4,income,0.726475,0.753937,0.637271,0.690712,"{0.0: 746, 1.0: 508}"
13,speeding_violations,0.708931,0.661202,0.805324,0.726182,"{0.0: 522, 1.0: 732}"
15,past_accidents,0.706539,0.649551,0.841930,0.733333,"{0.0: 475, 1.0: 779}"
6,vehicle_ownership,0.687400,0.729670,0.552413,0.628788,"{0.0: 799, 1.0: 455}"
8,married,0.649920,0.615057,0.720466,0.663602,"{0.0: 550, 1.0: 704}"
7,vehicle_year,0.636364,0.575286,0.921797,0.708440,"{0.0: 291, 1.0: 963}"
9,children,0.620415,0.641084,0.472546,0.544061,"{0.0: 811, 1.0: 443}"
10,postal_code,0.575758,0.579310,0.419301,0.486486,"{0.0: 819, 1.0: 435}"


Standard Vector Machine Classifier

 Best Feature: driving_experience (Accuracy: 0.7679)

 Summary of All Features:



,Feature,Accuracy,Precision,Recall,F1,Predicted_Classes
2,driving_experience,0.767943,0.779783,0.718802,0.748052,"{0.0: 700, 1.0: 554}"
0,age,0.706539,0.656376,0.813644,0.726597,"{0.0: 509, 1.0: 745}"
15,past_accidents,0.706539,0.649551,0.841930,0.733333,"{0.0: 475, 1.0: 779}"
6,vehicle_ownership,0.700159,0.756264,0.552413,0.638462,"{0.0: 815, 1.0: 439}"
5,credit_score,0.650718,0.648995,0.590682,0.618467,"{0.0: 707, 1.0: 547}"
13,speeding_violations,0.649920,0.591837,0.868552,0.703978,"{0.0: 372, 1.0: 882}"
7,vehicle_year,0.640351,0.578288,0.921797,0.710712,"{0.0: 296, 1.0: 958}"
4,income,0.637959,0.631016,0.589018,0.609294,"{0.0: 693, 1.0: 561}"
8,married,0.635566,0.599723,0.720466,0.654573,"{0.0: 532, 1.0: 722}"
9,children,0.631579,0.662005,0.472546,0.551456,"{0.0: 825, 1.0: 429}"
